In [34]:
!pip install pandas==0.24.2

In [35]:
!pip install scikit-learn==0.21.2

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb

In [37]:
if os.name=='posix':
    # GOOGLE COLAB SETUP

    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    #2. Get the file
    data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
    codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/AOBDL_TML'


    #3. Read file as panda dataframe
    train         = pd.read_csv(f'{data_path}/twitter_cleaned_no_punkt.csv') 
else:
        #2. Get the file
    data_path     = '../data'
    codes_path    = '../AOBDL_TML'


    #3. Read file as panda dataframe
    train         = pd.read_csv(f'{data_path}/twitter_cleaned_no_punkt.csv') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train.drop(['count', 'hate_speech', 'offensive_language', 'neither', 'class'], axis=1, inplace=True)
train.tweet.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['tweet', 'mal'])

In [0]:
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.tweet, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

In [40]:
print(f'X {X.shape}; X_test {X_test.shape}')

X (17549,); X_test (7168,)


In [0]:
# Ridge

In [0]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fscore_ = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    #stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 10000)

In [0]:
#auc_pr = 0.093434375824395
#auc_roc = 0.02343524395
c = 0
C_parameter = np.arange(0.1, 1.1, 0.1) 

# use best C

C_parameter = [1]

In [44]:
for c_p in C_parameter:  
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = LogisticRegression(C=c_p, solver='sag', random_state=rs)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            threshold = 0.7
            #for threshold in np.arange(0.1, 1, 0.1):
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            if len(C_parameter)==1:
                # print performance
                print(f'---------------------------------------------')
                print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
                print(f'---------------------------------------------')
                print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            fscore_.append(fscore)

      if len(C_parameter)!=1:
             print(f'PARAMETER C = {c_p}')

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

      print(f'-----------------------------------------------')
      print(f'CV std: AUC PR-C = {round(np.array(auc).std(), 3)}, AUC ROC = {round(np.array(roc).std(), 3)}, FSCORE  = {round(np.array(fscore_).std(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

 0.7 fold 0 precision 0.981 recall 0.944 fscore 0.962
---------------------------------------------
FOLD 0: AUC PR-C = 0.996, AUC ROC = 0.979
---------------------------------------------

 0.7 fold 1 precision 0.983 recall 0.952 fscore 0.967
---------------------------------------------
FOLD 1: AUC PR-C = 0.996, AUC ROC = 0.981
---------------------------------------------

 0.7 fold 2 precision 0.98 recall 0.951 fscore 0.965
---------------------------------------------
FOLD 2: AUC PR-C = 0.995, AUC ROC = 0.978
---------------------------------------------

 0.7 fold 3 precision 0.98 recall 0.951 fscore 0.965
---------------------------------------------
FOLD 3: AUC PR-C = 0.995, AUC ROC = 0.976
---------------------------------------------

 0.7 fold 4 precision 0.984 recall 0.948 fscore 0.966
---------------------------------------------
FOLD 4: AUC PR-C = 0.996, AUC ROC = 0.98
---------------------------------------------

-----------------------------------------------
CV average

In [45]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = LogisticRegression(C=1, solver='sag', random_state=rs)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.7
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.9952, AUC ROC = 0.978
-----------------------------------------

 0.7 precision 0.982 recall 0.947 fscore 0.964


In [0]:
# RF

In [0]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fscore_ = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 10000)

In [48]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20, random_state=rs)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            
            threshold = 0.7
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

print(f'-----------------------------------------------')
print(f'CV std: AUC PR-C = {round(np.array(auc).std(), 3)}, AUC ROC = {round(np.array(roc).std(), 3)}, FSCORE  = {round(np.array(fscore_).std(), 3)}')
print(f'-----------------------------------------------')
print(f'')

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    8.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   20.3s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   29.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.9s finished


---------------------------------------------
FOLD 0: AUC PR-C = 0.995, AUC ROC = 0.978
---------------------------------------------

 0.7 fold 0 precision 0.985 recall 0.937 fscore 0.961


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    8.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   20.1s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   28.9s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.9s finished


---------------------------------------------
FOLD 1: AUC PR-C = 0.996, AUC ROC = 0.98
---------------------------------------------

 0.7 fold 1 precision 0.986 recall 0.94 fscore 0.963


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    8.0s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   20.0s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   28.7s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.9s finished


...

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    8.0s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   20.1s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   28.5s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.9s finished


---------------------------------------------
FOLD 3: AUC PR-C = 0.994, AUC ROC = 0.975
---------------------------------------------

 0.7 fold 3 precision 0.981 recall 0.946 fscore 0.963


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    8.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   20.2s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   29.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.6s


---------------------------------------------
FOLD 4: AUC PR-C = 0.995, AUC ROC = 0.979
---------------------------------------------

 0.7 fold 4 precision 0.987 recall 0.944 fscore 0.965
-----------------------------------------------
CV average: AUC PR-C = 0.995, AUC ROC = 0.978, FSCORE  = 0.963
-----------------------------------------------

-----------------------------------------------
CV std: AUC PR-C = 0.001, AUC ROC = 0.002, FSCORE  = 0.001
-----------------------------------------------



[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.9s finished


In [49]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20, random_state=rs)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.7
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   10.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   27.0s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   38.6s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    1.2s


-----------------------------------------
TEST: AUC PR-C = 0.995, AUC ROC = 0.978
-----------------------------------------

 0.7 precision 0.984 recall 0.94 fscore 0.962


[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.6s finished


In [0]:
# SVM

In [57]:
penalty = ['l2', 'l1']
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
max_iter = [1000, 10000, 15000]

# best parameters
penalty = ['l1']
alpha = [1e-05]
max_iter = [1000] 

for p in penalty:
  for a in alpha:
    for i in max_iter:
      auc = []
      roc = []
      fscore_ = []
      c = 0
      # print performance
      print(f'-------------')
      print(f'penalty {p}')
      print(f'alpha {a}')
      print(f'max_iter {i}')
      print(f'-------------')
      
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=p, alpha=a, max_iter=i)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            threshold = 0.7
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

      print(f'-----------------------------------------------')
      print(f'CV std: AUC PR-C = {round(np.array(auc).std(), 3)}, AUC ROC = {round(np.array(roc).std(), 3)}, FSCORE  = {round(np.array(fscore_).std(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

-------------
penalty l1
alpha 1e-05
max_iter 1000
-------------
---------------------------------------------
FOLD 0: AUC PR-C = 0.997, AUC ROC = 0.984
---------------------------------------------

 0.7 fold 0 precision 0.987 recall 0.958 fscore 0.972
---------------------------------------------
FOLD 1: AUC PR-C = 0.997, AUC ROC = 0.983
---------------------------------------------

 0.7 fold 1 precision 0.986 recall 0.958 fscore 0.972
---------------------------------------------
FOLD 2: AUC PR-C = 0.995, AUC ROC = 0.982
---------------------------------------------

 0.7 fold 2 precision 0.984 recall 0.964 fscore 0.974
---------------------------------------------
FOLD 3: AUC PR-C = 0.995, AUC ROC = 0.979
---------------------------------------------

 0.7 fold 3 precision 0.984 recall 0.96 fscore 0.972
---------------------------------------------
FOLD 4: AUC PR-C = 0.997, AUC ROC = 0.983
---------------------------------------------

 0.7 fold 4 precision 0.986 recall 0.955 fsco

In [52]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=penalty[0], alpha=alpha[0], max_iter=max_iter[0])
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.7
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.996, AUC ROC = 0.982
-----------------------------------------

 0.7 precision 0.983 recall 0.957 fscore 0.97


In [0]:
# lightgbm

In [0]:
# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         100,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          20,
    'random_state':      rs,
}

In [55]:
auc = []
roc = []
fscore_ = []
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = lgb.LGBMClassifier(**lgb_params) 
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            threshold = 0.5
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

print(f'-----------------------------------------------')
print(f'CV std: AUC PR-C = {round(np.array(auc).std(), 3)}, AUC ROC = {round(np.array(roc).std(), 3)}, FSCORE  = {round(np.array(fscore_).std(), 3)}')
print(f'-----------------------------------------------')
print(f'')

---------------------------------------------
FOLD 0: AUC PR-C = 0.995, AUC ROC = 0.978
---------------------------------------------

 0.5 fold 0 precision 0.981 recall 0.958 fscore 0.969
---------------------------------------------
FOLD 1: AUC PR-C = 0.996, AUC ROC = 0.981
---------------------------------------------

 0.5 fold 1 precision 0.976 recall 0.963 fscore 0.969
---------------------------------------------
FOLD 2: AUC PR-C = 0.995, AUC ROC = 0.98
---------------------------------------------

 0.5 fold 2 precision 0.977 recall 0.964 fscore 0.971
---------------------------------------------
FOLD 3: AUC PR-C = 0.995, AUC ROC = 0.977
---------------------------------------------

 0.5 fold 3 precision 0.975 recall 0.964 fscore 0.969
---------------------------------------------
FOLD 4: AUC PR-C = 0.996, AUC ROC = 0.982
---------------------------------------------

 0.5 fold 4 precision 0.98 recall 0.965 fscore 0.972
-----------------------------------------------
CV averag

In [56]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = lgb.LGBMClassifier(**lgb_params) 
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.5
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.996, AUC ROC = 0.98
-----------------------------------------

 0.5 precision 0.976 recall 0.962 fscore 0.969
